### What is Spark?
Apache Spark is an open-source distributed general-purpose cluster-computing framework. Spark comes into picture when the user is working with large datasets that do not fit into the local machine.

On a high level, it is a unified analytics engine for Big Data processing, with built-in modules for streaming, SQL, machine learning, and graph processing. Spark is one of the latest technologies that is being used to quickly and easily handle Big Data and can interact with language shells like Scala, Python, and R.

### Selecting a sample dataset

In [0]:
# import the necessary libraries
import pyspark
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import col

In this notebook, we will be selecting a sample dataset that comes with Databricks. To view the different sample datasets, run the below cell.

In [0]:
# A list of sample datasets ready to be used
display(dbutils.fs.ls("/databricks-datasets/samples/"))

path,name,size
dbfs:/databricks-datasets/samples/adam/,adam/,0
dbfs:/databricks-datasets/samples/data/,data/,0
dbfs:/databricks-datasets/samples/docs/,docs/,0
dbfs:/databricks-datasets/samples/lending_club/,lending_club/,0
dbfs:/databricks-datasets/samples/newsgroups/,newsgroups/,0
dbfs:/databricks-datasets/samples/people/,people/,0
dbfs:/databricks-datasets/samples/population-vs-price/,population-vs-price/,0


### Exploring the data

Perform basic data exploration on population versus price sample dataset.

In [0]:
# Loading in a sample table into the dataframe
df = spark.read.csv("/databricks-datasets/samples/population-vs-price/data_geo.csv", header=True)

`df.head()` is another alternative to `df.show()`. By default, .head() lists the top 5 rows and .show() lists the first 20.

.head() is a Python Pandas function while .show() is a PySpark function.

In [0]:
# Viewing the dataframe
df.show(5)

+---------+-------------+-------+----------+------------------------+-----------------------+
2014 rank| City| State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
 101| Birmingham|Alabama| AL| 212247| 162.9|
 125| Huntsville|Alabama| AL| 188226| 157.7|
 122| Mobile|Alabama| AL| 194675| 122.5|
 114| Montgomery|Alabama| AL| 200481| 129|
 64|Anchorage[19]| Alaska| AK| 301010| null|
+---------+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows

To view the column names within the dataframe, call `df.columns`. This will return a list of the column names within the dataframe

In [0]:
# Viewing the column names
df.columns

Out[33]: ['2014 rank',
 'City',
 'State',
 'State Code',
 '2014 Population estimate',
 '2015 median sales price']

Notice that many of the column names contain spaces; this is not ideal if we want to implement SQL to create queries from this dataframe. To change the column names, we can implement the `.withColumnRenamed()` method

In [0]:
df2 = df.withColumnRenamed('2014 rank', '2014_rank')\
.withColumnRenamed('State Code', 'state_code')\
.withColumnRenamed('2014 Population estimate', '2014_pop_estimate')\
.withColumnRenamed('2015 median sales price', '2015_median_sales_price')

A new dataframe with variable (df2) was created to hold these changes. If we were to simply use `df.withColumnRenamed…`, (as we did above) it would only be a temporary change — there is no “inplace=True” parameter. We can also chain these all at once for each column name we want to be changed.

In [0]:
# viewing the numerical details for each column
df2.describe().show()

+-------+-----------------+-------+---------+----------+------------------+-----------------------+
summary| 2014_rank| City| State|state_code| 2014_pop_estimate|2015_median_sales_price|
+-------+-----------------+-------+---------+----------+------------------+-----------------------+
 count| 294| 294| 294| 294| 293| 109|
 mean| 147.5| null| null| null|307284.89761092153| 211.2660550458717|
 stddev|85.01470461043782| null| null| null| 603487.8272175139| 134.017245449277|
 min| 1|Abilene| Alabama| AK| 101408| 105.2|
 max| 99|Yonkers|Wisconsin| WI| 912791| 95.4|
+-------+-----------------+-------+---------+----------+------------------+-----------------------+

In [0]:
# Selecting the columns to view. If the dataframe had more than two columns, we would only be seeing the targeted columns in the .select() method
df2.select(['2014_rank', '2014_pop_estimate']).show()

+---------+-----------------+
2014_rank|2014_pop_estimate|
+---------+-----------------+
 101| 212247|
 125| 188226|
 122| 194675|
 114| 200481|
 64| 301010|
 78| 254276|
 86| 239277|
 88| 237517|
 38| 464704|
 148| 166934|
 6| 1537058|
 95| 230512|
 215| 126275|
 142| 172816|
 33| 527972|
 119| 197706|
 56| 346997|
 261| 108930|
 52| 368759|
 227| 118853|
+---------+-----------------+
only showing top 20 rows

To view the types of values within each column, use `.printSchema()` method on df2 dataframe

In [0]:
# Printing out the schema of the dataframe
df2.printSchema()

root
-- 2014_rank: string (nullable = true)
-- City: string (nullable = true)
-- State: string (nullable = true)
-- state_code: string (nullable = true)
-- 2014_pop_estimate: string (nullable = true)
-- 2015_median_sales_price: string (nullable = true)

Notice that all of the columns contain string values — even the columns that are supposed to be numerical values! 

A way we can manually adjust the type of values within a column is similar to adjusting the names of the columns using `.withColumn()` method and chaining on the `.cast()` method

Similar to the previous method, “.withColumnRenamed()”, the change is only temporary unless a new dataframe is created to hold the changes. Again, we can chain multiple columns at once.

In [0]:
df3 = df2.withColumn("2014_rank",col("2014_rank").cast(IntegerType())) \
       .withColumn("2014_pop_estimate", col("2014_pop_estimate").cast(IntegerType())) \
       .withColumn('2015_median_sales_price', col('2015_median_sales_price').cast(FloatType()))

In [0]:
df3.printSchema()

root
-- 2014_rank: integer (nullable = true)
-- City: string (nullable = true)
-- State: string (nullable = true)
-- state_code: string (nullable = true)
-- 2014_pop_estimate: integer (nullable = true)
-- 2015_median_sales_price: float (nullable = true)

In [0]:
df3.show(5)

+---------+-------------+-------+----------+-----------------+-----------------------+
2014_rank| City| State|state_code|2014_pop_estimate|2015_median_sales_price|
+---------+-------------+-------+----------+-----------------+-----------------------+
 101| Birmingham|Alabama| AL| 212247| 162.9|
 125| Huntsville|Alabama| AL| 188226| 157.7|
 122| Mobile|Alabama| AL| 194675| 122.5|
 114| Montgomery|Alabama| AL| 200481| 129.0|
 64|Anchorage[19]| Alaska| AK| 301010| null|
+---------+-------------+-------+----------+-----------------+-----------------------+
only showing top 5 rows

### Using SQL syntax

Before ending the data analysis, let us try SQL querying on the dataframe created above. 

For SQL to work well, we need to make sure `df3` dataframe has a table name. To do this, simply create a temporary view.

In [0]:
df3.createOrReplaceTempView('pop_price')

Execute the SQL query

In [0]:
# We can also use SQL to query information in the dataframe and create a new dataframe with the queried results
top_10_results = spark.sql("""SELECT * FROM pop_price 
                           WHERE 2014_rank <= 10
                           SORT BY 2014_rank ASC""")
top_10_results.show()

+---------+---------------+------------+----------+-----------------+-----------------------+
2014_rank| City| State|state_code|2014_pop_estimate|2015_median_sales_price|
+---------+---------------+------------+----------+-----------------+-----------------------+
 1| New York[6]| New York| NY| 8491079| 388.6|
 2| Los Angeles| California| CA| 3928864| 434.7|
 3| Chicago| Illinois| IL| 2722389| 192.5|
 4| Houston[7]| Texas| TX| 2239558| 200.3|
 5|Philadelphia[8]|Pennsylvania| PA| 1560297| 204.9|
 6| Phoenix| Arizona| AZ| 1537058| 206.1|
 7| San Antonio| Texas| TX| 1436697| 184.7|
 8| San Diego| California| CA| 1381069| 510.3|
 9| Dallas| Texas| TX| 1281047| 192.5|
 10| San Jose| California| CA| 1015785| 900.0|
+---------+---------------+------------+----------+-----------------+-----------------------+